# Fine pruning defense

This notebook will contain code to run the fine prining defense https://link.springer.com/chapter/10.1007/978-3-030-00470-5_13

In [1]:
import os

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16

from kerassurgeon import Surgeon
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels

from sklearn.model_selection import train_test_split

## Data

In [4]:
trn_x = np.load('../../65/trn_x.npy')
trn_y = np.load('../../65/trn_y.npy')
pt_x = np.load('../../65/pt_x.npy')
pt_y = np.load('../../65/pt_y.npy')
p_x = np.load('../../65/p_x.npy')
p_y = np.load('../../65/p_y.npy')
tst_x = np.load('../../65/tst_x.npy')
tst_y = np.load('../../65/tst_y.npy')
valid_x = np.load('../../65/valid_x.npy')
valid_y = np.load('../../65/valid_y.npy')

valid_x, ft_x, valid_y, ft_y = train_test_split(valid_x, valid_y, test_size=0.1, random_state=42)

print(
    '\ntrain',
    trn_x.shape,
    trn_y.shape,
    '\npt',
    pt_x.shape,
    pt_y.shape,
    '\np',
    p_x.shape,
    p_y.shape,
    '\ntst',
    tst_x.shape,
    tst_y.shape,
    '\nft',
    ft_x.shape,
    ft_y.shape,
    '\nvalid',
    valid_x.shape,
    valid_y.shape,
)


train (7158, 100, 100, 3) (7158, 2) 
pt (42, 100, 100, 3) (42, 2) 
p (158, 100, 100, 3) (158, 2) 
tst (6054, 100, 100, 3) (6054, 2) 
ft (700, 100, 100, 3) (700, 2) 
valid (6300, 100, 100, 3) (6300, 2)


## Model

Placeholder

In [5]:
model = keras.models.load_model('../../65/model')

In [6]:
score = model.evaluate(tst_x, tst_y, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
original_acc = score[1]

Test loss: 0.4282311201095581
Test accuracy: 0.83927983045578


In [7]:
score_pt = model.evaluate(pt_x, pt_y, verbose=0)
print("pt loss:", score_pt[0])
print("pt accuracy:", score_pt[1])

pt loss: 0.6295231580734253
pt accuracy: 0.6190476417541504


In [8]:
print(model.count_params())
print(model.summary())

14715714
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 1

## Pruning

In [9]:
stop_cond = False
cur_idx = 0
tgt_layer = 'block5_conv3'

target_layer = model.get_layer(tgt_layer)
apoz = identify.get_apoz(model, target_layer, tst_x)
print('Neurons in layer', len(apoz))  
    
high_apoz_channels = identify.high_apoz(apoz, "both", cutoff_absolute=0.55)
if len(high_apoz_channels) == 0: 
    print('Zero channels found')
        
while not stop_cond:
    target_layer = model.get_layer(tgt_layer)  
    print('Neurons to prune', high_apoz_channels[cur_idx])
    
    model = delete_channels(model, target_layer, [high_apoz_channels[cur_idx],])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    score = model.evaluate(tst_x, tst_y, verbose=0)

    print('Number of neurons pruned:', cur_idx)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    cur_idx += 1
    if original_acc - score[1] > .04:
        stop_cond = True
#     keras.backend.clear_session()
        
print('\nFINAL')
print('Number of weights set to 0:', cur_idx)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Neurons in layer 512
Neurons to prune 0
Deleting 1/512 channels from layer: block5_conv3
Number of neurons pruned: 0
Test loss: 0.42780521512031555
Test accuracy: 0.8391146063804626
Neurons to prune 1
Deleting 1/511 channels from layer: block5_conv3
Number of neurons pruned: 1
Test loss: 0.427564799785614
Test accuracy: 0.8396101593971252
Neurons to prune 2
Deleting 1/510 channels from layer: block5_conv3
Number of neurons pruned: 2
Test loss: 0.42717891931533813
Test accuracy: 0.8404361009597778
Neurons to prune 3
Deleting 1/509 channels from layer: block5_conv3
Number of neurons pruned: 3
Test loss: 0.42717885971069336
Test accuracy: 0.8402708768844604
Neurons to prune 4
Deleting 1/508 channels from layer: block5_conv3
Number of neurons pruned: 4
Test loss: 0.4289904534816742
Test accuracy: 0.83894944190979
Neurons to prune 5
Deleting 1/507 channels from layer: block5_conv3
Number of neurons pruned: 5
Test loss: 0.4276634752750397
Test accuracy: 0.8391146063804626
Neurons to prune 6


Number of neurons pruned: 51
Test loss: 0.4289844036102295
Test accuracy: 0.8412619829177856
Neurons to prune 52
Deleting 1/460 channels from layer: block5_conv3
Number of neurons pruned: 52
Test loss: 0.428962767124176
Test accuracy: 0.8412619829177856
Neurons to prune 53
Deleting 1/459 channels from layer: block5_conv3
Number of neurons pruned: 53
Test loss: 0.4290950894355774
Test accuracy: 0.8414271473884583
Neurons to prune 54
Deleting 1/458 channels from layer: block5_conv3
Number of neurons pruned: 54
Test loss: 0.42934706807136536
Test accuracy: 0.8409315943717957
Neurons to prune 55
Deleting 1/457 channels from layer: block5_conv3
Number of neurons pruned: 55
Test loss: 0.43009769916534424
Test accuracy: 0.8401057124137878
Neurons to prune 57
Deleting 1/456 channels from layer: block5_conv3
Number of neurons pruned: 56
Test loss: 0.4303472638130188
Test accuracy: 0.8404361009597778
Neurons to prune 58
Deleting 1/455 channels from layer: block5_conv3
Number of neurons pruned: 5

In [10]:
print(model.count_params())
print(model.summary())

14356056
Model: "functional_157"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25,

In [11]:
score = model.evaluate(tst_x, tst_y, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.49921074509620667
Test accuracy: 0.7819623351097107


In [12]:
score_pt = model.evaluate(pt_x, pt_y, verbose=0)
print("pt loss:", score_pt[0])
print("pt accuracy:", score_pt[1])

pt loss: 0.8384241461753845
pt accuracy: 0.523809552192688


In [13]:
model.optimizer.learning_rate.assign(0.0001)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
model.fit(ft_x, ft_y, batch_size=32, epochs=5)

Epoch 1/5
 2/22 [=>............................] - ETA: 0s - loss: 2.5576 - accuracy: 0.6719WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0230s vs `on_train_batch_end` time: 0.0361s). Check your callbacks.


22/22 [==============================] - 2s 78ms/step - loss: 0.8803 - accuracy: 0.5514
Epoch 2/5
22/22 [==============================] - 1s 57ms/step - loss: 0.5605 - accuracy: 0.7357
Epoch 3/5
22/22 [==============================] - 1s 57ms/step - loss: 0.4342 - accuracy: 0.8229
Epoch 4/5
22/22 [==============================] - 1s 57ms/step - loss: 0.2724 - accuracy: 0.9157
Epoch 5/5
22/22 [==============================] - 1s 57ms/step - loss: 0.1507 - accuracy: 0.9529


In [15]:
score = model.evaluate(tst_x, tst_y, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.3276580274105072
Test accuracy: 0.9038652181625366


In [16]:
score_pt = model.evaluate(pt_x, pt_y, verbose=0)
print("pt loss:", score_pt[0])
print("pt accuracy:", score_pt[1])

pt loss: 0.6585677266120911
pt accuracy: 0.8095238208770752
